Import Libraries

In [405]:
from pyvis.network import Network
import networkx as nx
import random
from networkx.algorithms.community import greedy_modularity_communities
from networkx.linalg.modularitymatrix import modularity_matrix, directed_modularity_matrix
import numpy as np

Create Helper Functions

In [406]:
# recursive function that will traverse the nodes
def createPaper(network, authors, probStop):
    '''
    Will take network, list of authors, and probStop as input
    '''
    currAuthorID = authors[-1]
    newNeighbors = set(network.neighbors(currAuthorID)).difference(set(authors))

    # base condition: stop at node if probStop hit or there are no new neighbors to traverse
    if random.random() < probStop or len(newNeighbors) == 0:
        return
    
    # create list reprsenting probabilities for the neighboring nodes of the current coauthor
    probs = []
    for neighbor in newNeighbors:
        nData = network.get_edge_data(currAuthorID, neighbor)
        probs.extend([neighbor] * nData["weight"])

    # Select coauthor from neighbors probabilities list
    coauthorID = random.choice(probs)

    # update all edges of coauthors to this new author
    for author in authors:
        # if there is not an edge, create one
        if not network.has_edge(author, coauthorID) and author != coauthorID:
            network.add_edge(author, coauthorID, weight=0, width=1)
        newWeight = network.get_edge_data(author, coauthorID)["weight"] + 1
        #network.update(edges=[ (author, coauthorID, {"weight": newWeight, "width": newWeight//2}) ])
        network.update(edges=[ (author, coauthorID, {"weight": newWeight}) ])

    # call function recursively with coauthor
    authors.append(coauthorID)
    createPaper(network, authors, probStop)


Define initial parameters

In [407]:
# define time steps
timeSteps = 15

# Probabilities
# probability that you generate new author
probNewAuthor = 0.5
# probability that you stop at a given node
probStop = 0.7

# define fields
fieldColors = {"CS": "blue", 
                "Math": "green", 
                "Physics": "red"}
fields = list(fieldColors.keys())

# define initial scholars, will be in form (id, scholarField, color)
scholarField = random.choice(fields)
nodeID = 0


Create Model

In [408]:
network = nx.Graph()
scholarField = random.choice(fields)
network.add_node(nodeID, label=scholarField, color=fieldColors[scholarField])
# go through time steps, add new scholar and paper at each step
for i in range(1, timeSteps):

    # Choose first author, either new scholar or random choice
    currNodes = list(network.nodes())
    authors = [random.choice(currNodes)]

    # with probability, add new author to network set as main author with a coauthor
    if random.random() < probNewAuthor:
        # generate author and field
        scholarField = random.choice(fields)
        nodeID += 1
        author = nodeID
        network.add_node(author, label=scholarField, color=fieldColors[scholarField])

        # generate random coauthor from currNodes, which doesn't have the new node added in
        coauthorID = random.choice(currNodes)
        network.add_edge(author, coauthorID, weight=1, width=1)

        # update authors list
        authors = [author, coauthorID]

    # Add new paper, calling function
    createPaper(network, authors, probStop)

Display Network

In [409]:
nt = Network()
# populates the nodes and edges data structures
nt.from_nx(network)
print(network.edges.data())
nt.show('docs/models/modularity.html')

[(0, 1, {'weight': 2, 'width': 1}), (0, 2, {'weight': 4, 'width': 1}), (0, 5, {'weight': 1, 'width': 1}), (0, 3, {'weight': 2, 'width': 1}), (0, 6, {'weight': 1, 'width': 1}), (0, 7, {'weight': 1, 'width': 1}), (2, 3, {'weight': 3, 'width': 1}), (2, 6, {'weight': 1, 'width': 1}), (2, 7, {'weight': 1, 'width': 1}), (3, 4, {'weight': 1, 'width': 1}), (3, 6, {'weight': 1, 'width': 1})]


### Show modularity communities

In [410]:
communities = greedy_modularity_communities(network)
print(communities)
matrix = modularity_matrix(network)
print(matrix)

[frozenset({0, 1, 5}), frozenset({3, 4, 6}), frozenset({2, 7})]
[[-1.63636364  0.72727273 -0.09090909 -0.09090909 -0.27272727  0.72727273
   0.18181818  0.45454545]
 [ 0.72727273 -0.04545455 -0.18181818 -0.18181818 -0.04545455 -0.04545455
  -0.13636364 -0.09090909]
 [-0.09090909 -0.18181818 -0.72727273  0.27272727 -0.18181818 -0.18181818
   0.45454545  0.63636364]
 [-0.09090909 -0.18181818  0.27272727 -0.72727273  0.81818182 -0.18181818
   0.45454545 -0.36363636]
 [-0.27272727 -0.04545455 -0.18181818  0.81818182 -0.04545455 -0.04545455
  -0.13636364 -0.09090909]
 [ 0.72727273 -0.04545455 -0.18181818 -0.18181818 -0.04545455 -0.04545455
  -0.13636364 -0.09090909]
 [ 0.18181818 -0.13636364  0.45454545  0.45454545 -0.13636364 -0.13636364
  -0.40909091 -0.27272727]
 [ 0.45454545 -0.09090909  0.63636364 -0.36363636 -0.09090909 -0.09090909
  -0.27272727 -0.18181818]]


In [411]:
# get eigenvalues and vectors of matrix
eVals, eVects = np.linalg.eig(matrix)
# find indices of largest eigenvalue, get corresponding eigenvector
# if no positive eigenvalue, then cannot divide network further because it will not improve modularity
if max(eVals) > 0:
    index = np.argmax(eVals)
    maxEvect = np.array(eVects)[index]
    # split eigenvector into positive and negative values, collecting vertices
    g1 = []
    g2 = []
    for j, val in enumerate(maxEvect):
        if val < 0:
            g1.append(j)
            # test update the color
            network.update(nodes=[(j, {"label": "g1", "color": "green"})])
        else:
            g2.append(j)
            network.update(nodes=[(j, {"label": "g2", "color": "red"})])
print(g1)
print(g2)


[3, 7]
[0, 1, 2, 4, 5, 6]


In [412]:
# recreate network
nt = Network()
# populates the nodes and edges data structures
nt.from_nx(network)
print(network.edges.data())
nt.show('docs/models/modularity.html')

[(0, 1, {'weight': 2, 'width': 1}), (0, 2, {'weight': 4, 'width': 1}), (0, 5, {'weight': 1, 'width': 1}), (0, 3, {'weight': 2, 'width': 1}), (0, 6, {'weight': 1, 'width': 1}), (0, 7, {'weight': 1, 'width': 1}), (2, 3, {'weight': 3, 'width': 1}), (2, 6, {'weight': 1, 'width': 1}), (2, 7, {'weight': 1, 'width': 1}), (3, 4, {'weight': 1, 'width': 1}), (3, 6, {'weight': 1, 'width': 1})]
